In [9]:
pip install "weaviate-client==3.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 7.3 MB/s eta 0:00:00


In [39]:
import weaviate

client = weaviate.Client("http://35.235.126.154:8080")


In [102]:
import requests
import base64

# URL of the image
image_url = "http://assets.myntassets.com/v1/images/style/properties/dce310e4c15223a6c964631190263284_images.jpg"

# Retrieve the image data
response = requests.get(image_url)
# print(response.content)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Encode the image data to base64
    base64_image = base64.b64encode(response.content).decode('utf-8')
    print(base64_image)
else:
    print("")


/9j/4AAQSkZJRgABAQIAHAAcAAD/2wCEAAkJCQkKCQoMDAoPEA4QDxUUEhIUFSAXGRcZFyAxHyQfHyQfMSw1KygrNSxOPTc3PU5aTEhMWm5iYm6Kg4q0tPIBCQkJCQoJCgwMCg8QDhAPFRQSEhQVIBcZFxkXIDEfJB8fJB8xLDUrKCs1LE49Nzc9TlpMSExabmJiboqDirS08v/CABEIBaAEOAMBIgACEQEDEQH/xAAzAAEAAgIDAQAAAAAAAAAAAAAAAQIDBQQHCAYBAQEBAQEBAAAAAAAAAAAAAAABAgMEBf/aAAwDAQACEAMQAAAA7xAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [51]:
# client.schema.delete_class("TestImage")
schema = {
   "classes": [
       {
          "class": "TestImage",
    "description": "Each object contains product image",
    "vectorizer": "img2vec-neural",
    "moduleConfig": {
      "img2vec-neural": {
        "imageFields": [
          "image"
        ]
      }
    },
    "properties": [
      {
        "dataType": [
          "blob"
        ],
        "description": "Product Image",
        "name": "image"
      },
      {
        "dataType": [
          "number"
        ],
        "description": "Unique Id of the product",
        "name": "ProductId"
      },
      {
        "dataType": [
          "text"
        ],
        "description": "Gender associated with the product",
        "name": "Gender"
      },
      {
        "dataType": [
          "text"
        ],
        "name": "Category"
      },
      {
        "dataType": [
          "text"
        ],
        "name": "SubCategory"
      },
      {
        "dataType": [
          "text"
        ],
        # "description": "Gender associated with the product",
        "name": "Colour"
      },
      {
        "dataType": [
          "text"
        ],
        # "description": "Gender associated with the product",
        "name": "ProductTitle"
      },
    ]
       }
   ]
}


client.schema.create(schema)
# print(res)

None


In [ ]:

# client.batch.configure(batch_size=100)  # Configure batch
# with client.batch as batch:  # Initialize a batch process
#     for i, d in enumerate(data):  # Batch import data
#         print(f"importing question: {i+1}")

#         batch.add_data_object(
#             data_object=properties,
#             class_name="Question"
#         )


In [103]:
properties = {
            "image":base64_image,
            "ProductId":42419	 ,
            "Gender":"Girls" ,
            "Category":"Apparel" ,
            "SubCategory":"Topwear",
            "Colour":"White",
            "ProductTitle":"Gini and Jony Girls Knit White Top	",
        }

In [104]:
# Prepare a batch process
client.batch.configure(batch_size=1)  # Configure batch
with client.batch as batch:
    # Batch import all Questions
    # for i, d in enumerate(data):
        # print(f"importing question: {i+1}")  # To see imports

      properties = properties

      batch.add_data_object(properties, "TestImage")

In [107]:
sourceImage = { "image":"/content/f3964f76c78edd85f4512d98b26d52e9_images.webp"}
result = (
  client.query
  .get("TestImage", ["image","gender","category","colour"])
  .with_near_image(sourceImage)
  .with_limit(1)

  .do()
)

print(result)

{'data': {'Get': {'TestImage': [{'category': 'Apparel', 'colour': 'White', 'gender': 'Girls', 'image': '...'}]}}}


In [108]:

sourceImage = { "image": base64_image}
result = (
  client.query
  .get("TestImage", ["image","gender","category","colour"])
  .with_near_image(sourceImage,encode=False)
  .with_limit(1)

  .do()
)

print(result)

{'data': {'Get': {'TestImage': [{'category': 'Apparel', 'colour': 'White', 'gender': 'Girls', 'image': '/9j/4AAQSkZJRgABAQIAHAAcAAD/2wCEAAkJCQkKCQoMDAoPEA4QDxUUEhIUFSAXGRcZFyAxHyQfHyQfMSw1KygrNSxOPTc3PU5aTEhMWm5iYm6Kg4q0tPIBCQkJCQoJCgwMCg8QDhAPFRQSEhQVIBcZFxkXIDEfJB8fJB8xLDUrKCs1LE49Nzc9TlpMSExabmJiboqDirS08v/CABEIBaAEOAMBIgACEQEDEQH/xAAzAAEAAgIDAQAAAAAAAAAAAAAAAQIDBQQHCAYBAQEBAQEBAAAAAAAAAAAAAAABAgMEBf/aAAwDAQACEAMQAAAA7xAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA